In [1]:
import sys
sys.path.append('/opt/miniconda3/lib/python3.7/site-packages')
# from urllib.request import urlopen
# import urllib
# from urllib.error import HTTPError
# from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep

In [2]:
# 设置
# bilibili主页链接
brand = "星巴克"
link = "https://space.bilibili.com/497969625/video"
filename = "data/starbucksBilibili.csv"

In [3]:
# 向文件中写入数据
def writeData(file,content):
    with open(file,'a') as f: # 如果filename不存在会自动创建， 'w'表示写数据，写之前会清空文件中的原有数据！
        f.write(content)

In [4]:
def getDataForVideo(link,filename):
# 爬取单个视频数据，输入视频链接和储存数据的文档
# 标题--title  点赞--like  投币--coin  收藏--collect  转发--share   简介--info  标签  
    driver = webdriver.Chrome()
    driver.get(link)

    # 爬数据
    title = driver.find_element_by_xpath("//*[@id='viewbox_report']/h1/span").text
    like = driver.find_element_by_xpath("//*[@id='arc_toolbar_report']/div[1]/span[1]").text
    coin = driver.find_element_by_xpath("//*[@id='arc_toolbar_report']/div[1]/span[2]").text
    collect = driver.find_element_by_xpath("//*[@id='arc_toolbar_report']/div[1]/span[3]").text
    share = driver.find_element_by_xpath("//*[@id='arc_toolbar_report']/div[1]/span[4]").text
    info = driver.find_element_by_xpath("//*[@id='v_desc']/div[2]").text
    info = info.replace('\n', '，')

    # 写入数据
    get_a_result_like = 'project,name,'+title+',get_a_result,effect,like,'+ like +'\n'
    get_a_result_coin = 'project,name,'+title+',get_a_result,effect,coin,'+ coin +'\n'
    get_a_result_collect = 'project,name,'+title+',get_a_result,effect,collect,'+ collect +'\n'
    get_a_result_share = 'project,name,'+title+',get_a_result,effect,share,'+ share +'\n'
    have_feature_info = 'project,name,'+title+',have_feature,feature,info,'+ info +'\n'

    writeData(filename,get_a_result_like)
    writeData(filename,get_a_result_coin)
    writeData(filename,get_a_result_share)
    writeData(filename,have_feature_info)

    # 爬tags，写tags
    tags = driver.find_elements_by_xpath("//*[@id='v_tag']/ul/li")
    for i in range(0,len(tags)):
        tag = tags[i].text
        have_feature_tag = 'project,name,'+title+',have_feature,feature,tag,'+ tag +'\n'
        writeData(filename,have_feature_tag)

    driver.close()

In [5]:
# 爬取bilibili视频信息
# 以星巴克为例，修改up主主页链接爬取其他内容
driver = webdriver.Chrome()
driver.get(link)

videoNum = driver.find_element_by_xpath("//*[@id='page-video']/div/div[1]/div/div/ul/li[1]/span").text
videoNum = int(videoNum)
# print(videoNum)

#------------------找到需要的元素-----------#
# 品牌--brand  视频名称--title  视频时长--length 观看量--playtimes 
# 投放时间--puttime  视频链接--link

for i in range(0,videoNum):
    title = driver.find_element_by_xpath("//*[@id='submit-video-list']/ul[2]/li["+ str(i+1) + "]/a[2]").text
    length = driver.find_element_by_xpath("//*[@id='submit-video-list']/ul[2]/li[" + str(i+1) + "]/a[1]/span[1]").text
    playtimes = driver.find_element_by_xpath("//*[@id='submit-video-list']/ul[2]/li[" + str(i+1) + "]/div/span[1]").text
    puttime = driver.find_element_by_xpath("//*[@id='submit-video-list']/ul[2]/li[" + str(i+1) + "]/div/span[2]").text
    link = driver.find_element_by_xpath("//*[@id='submit-video-list']/ul[2]/li[" + str(i+1) + "]/a[1]").get_attribute("href")
    # print(link)
    getDataForVideo(link,filename)


#--------------------数据储存---------------#
# 写入数据到表格中
    have_project = 'brand,name,'+brand+',have_project,project,name,'+title+'\n'
    be_showed_on = 'project,name,'+title+',be_showed_on,platform,name,bilibili\n'
    have_feature_length = 'project,name,'+title+',have_feature,feature,length,'+ length +'\n'
    have_feature_puttime = 'project,name,'+title+',have_feature,feature,time,'+ puttime +'\n'
    have_feature_link = 'project,name,'+title+',have_feature,feature,link,'+ link +'\n'
    get_a_result_playtimes = 'project,name,'+title+',get_a_result,effect,play times,'+ playtimes +'\n'


    writeData(filename,have_project)
    writeData(filename,be_showed_on) 
    writeData(filename,have_feature_length) 
    writeData(filename,have_feature_puttime) 
    writeData(filename,have_feature_link) 
    writeData(filename,get_a_result_playtimes) 

driver.close()